# EXPLORATORY DATA ANALYSIS

This notebook contains the exploration process that lead to defining the final model for recommending offers. The steps that I have followed are:

- Retrieve data from data files.
- Basic exploration of the files.
- Data cleaning.
- Exploration of possible features for the classification model.
- First attempt of a K-Means classification model.
- Selection of the best parameters for the model.

The prototype application of the offer recommender is contained in the notebook called **recommender**.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.cluster import KMeans

# Settings

Files containing the data sets.

In [ ]:
FILE_SKILLS = "data/candidatetest_df_off_sk.csv"
FILE_OFFERS = "data/candidatetest_df_off_fixed.csv"

# Data retrieval

The provided offers file (candidatetest_df_off.csv) was not a proper CSV, so I had to perform a process to parse it. The process took several hours, so here I start working with the processed file directly. The notebook called **parse_offers_file** contains the code used to obtain it.

In [ ]:
def ReadOffers(file):
    '''
    Read the file containing the fixed offers data set.
    Args:
    - file: File with the fixed offers data set.
    Returns:
    - Pandas dataframe with tidy offers data set.
    '''
    data = pd.read_csv(file, delimiter = "|", encoding = "latin1", dtype = {"id": str})
    data = data.loc[data["requirement"] != "#Â¿NOMBRE?", :]
    data["studies"] = [str(s).replace(";", "") for s in data["studies"]]
    return data

offers = ReadOffers(FILE_OFFERS)
offers

In [ ]:
def ReadSkills(file):
    '''
    Read the file containing the original skills data set.
    Args:
    - file: File with the fixed skills data set.
    Returns:
    - Pandas dataframe with tidy skills data set.
    '''
    data = pd.read_csv(file, delimiter = "|", encoding = "latin1")
    data = data.rename(columns={"O_ID": "id", "OSK_NOMBRE": "skill"})
    return data

skills = ReadSkills(FILE_SKILLS)
skills

# Data exploration

## Offers dataset

In [ ]:
offersNum = len(set(offers["id"]))
namesNum = len(set(offers["name"]))
requirementsNum = len(set(offers["requirement"]))
studiesNum = len(set(offers["studies"]))
print("Ths offers dataset contains:")
print("- "+ format(offersNum, ",") + " distinct offers.")
print("- "+ format(namesNum, ",") + " distinct names.")
print("- "+ format(requirementsNum, ",") + " distinct requirements.")
print("- "+ format(studiesNum, ",") + " distinct studies.")

## Skills dataset

In [ ]:
offersNum = len(set(skills["id"]))
skillsNum = len(set(skills["skill"]))
print("Ths skills dataset contains:")
print("- "+ format(offersNum, ",") + " distinct offers.")
print("- "+ format(skillsNum, ",") + " distinct skills.")

## Comparing datasets

In [ ]:
offersIds = list(set(offers.loc[:, "id"]))
skillsIds = list(set(skills.loc[:, "id"]))
offers_in_skills_and_offers = sum(pd.Series(skillsIds).isin(offersIds))
offers_no_skills_in_offers = sum(~pd.Series(offersIds).isin(skillsIds))
offers_in_skills_no_offers = sum(~pd.Series(skillsIds).isin(offersIds))
print("Offers in both datasets: " + format(offers_in_skills_and_offers, ","))
print("Offers in the offers dataset but not in the skills dataset: " + format(offers_no_skills_in_offers, ","))
print("Offers in the skills dataset but not in the offers dataset: " + format(offers_in_skills_no_offers, ","))

I compare the data for one random id in both datasets:

In [ ]:
skills.loc[skills["id"] == "31854473284", ]

In [ ]:
offers.loc[offers["id"] == "31854473284", :]

The offers in both files do not seem to match.

## Cleaning offers

I group the studies in subgroups based on profiles of candidates that could be interested in the same kind of offers.

In [ ]:
print("Different studies:")
set(offers["studies"])

In [ ]:
def AddStudiesType(df):
    '''
    Add a column to the dataframe containing a broader type of studies.
    Args:
    - df: Pandas dataframe with a column called "studies".
    Returns:
    - Pandas dataframe with a new column called "studies_type".
    '''
    studiesDict = {"(Indicar Nivel)": "no indicado",
                   "Alta capacidad de trabajo en equipo y de relacion con otros grupos tecnicos\t\t": "no indicado",
                   "Bachillerato": "secundaria",
                   "Ciclo Formativo Gr": "fp",
                   "Ciclo Formativo Grado": "fp",
                   "Ciclo Formativo Grado Medio": "fp",
                   "Ciclo Formativo Grado Superior": "fp",
                   "Diplomatura": "universidad",
                   "Doctorado": "posgrado",
                   "Educacion ": "no idicado",
                   "Educacion Secundaria Obligatoria": "secundaria",
                   "Ense?anzas artisticas (regladas)": "fp",
                   "Ense?anzas deportivas (regladas)": "fp",
                   "Formaci": "no indicado",
                   "Formacion Profesion": "fp",
                   "Formacion Profesional Grado Me": "fp",
                   "Formacion Profesional Grado Medio": "fp",
                   "Formacion Profesional Grado S": "fp",
                   "Formacion Profesional Grado Super": "fp",
                   "Formacion Profesional Grado Superior": "fp",
                   "Gra": "universidad",
                   "Grado": "universidad",
                   "Ingenieria Superior": "universidad",
                   "Ingenieria Tecn": "universidad",
                   "Ingenieria Tecnica": "universidad",
                   "Licenciatura": "universidad",
                   "Master": "posgrado",
                   "Otros cursos y formacion no reglada": "no indicado",
                   "Otros titulos": "no indicado",
                   "Postgrado": "posgrado",
                   "Sin estudios": "sin estudios",
                   "nan": "no indicado"}
    df["studies_type"] = df["studies"].map(studiesDict)
    return df

offers = AddStudiesType(offers)

In [ ]:
print("Number of offers per type of studies:")
offers[["id", "studies_type"]].groupby("studies_type").count().reset_index()

## Cleaning skills

The process of cleaning skills consists on:
- Removing some patterns such as
    - Symbols working as a bullet points at the beginning.
    - Parentheses.
    - Meaningless words such as articles, prepositions, etc.
    - Blank spaces.
- Using only lower case.

In [ ]:
PATTERNS_BEGINNING = [" ", "+", "?", "#", "$", "-", "(", "*"]
PATTERNS_END = [" ", ")"]
STOP_WORDS = stopwords.words("spanish") + stopwords.words("english")

def CleanSkill(skill, patternsBeginning = [], patternsEnd = [], lowerCase = True, stopWords = []):
    '''
    Performe cleaning to a skill by removing meaningless parts and using only lowercase.
    Args:
    - skill: String with the skill to clean.
    - patternsBeginning: List of patterns to remove from the beginning of the skill.
    - patternsEnd: List of patterns to remove from the end of the skill.
    - stopWords: List of words to eliminate.
    Returns:
    - String with the skill clean.
    '''
    skillClean = str(skill)
    skillClean = skillClean.lstrip("".join(patternsBeginning))
    skillClean = skillClean.rstrip("".join(patternsEnd))
    if lowerCase == True:
        skillClean = skillClean.lower()
    skillClean = " ".join([x for x in skillClean.split(" ") if x not in stopWords])
    skillClean = skillClean.replace(" +", " ")
    skillClean = skillClean.lstrip()
    skillClean = skillClean.rstrip()
    return(skillClean)

def CleanSkills(df, patternsBeginning, patternsEnd, stopWords):
    '''
    Create a new column to a Pandas dataframe with clean skills.
    Args:
    - df: Pandas dataframe with a column called "skill".
    - patternsBeginning: List of patterns to remove from the beginning of the skill.
    - patternsEnd: List of patterns to remove from the end of the skill.
    - stopWords: List of words to eliminate.
    Returns:
    - Pandas dataframe with a new column called "skill_clean".
    '''
    df["skill_clean"] = df["skill"].apply(lambda x: CleanSkill(x,
                                                               patternsBeginning = patternsBeginning,
                                                               patternsEnd = patternsEnd,
                                                               lowerCase = True,
                                                               stopWords = stopWords))
    return df

skills = CleanSkills(skills, PATTERNS_BEGINNING, PATTERNS_END, STOP_WORDS)
skills

In [ ]:
cleanSkillsNum = len(set(skills["skill_clean"]))
print("Ths skills dataset contains:")
print("- "+ format(skillsNum, ",") + " distinct skills.")
print("- "+ format(cleanSkillsNum, ",") + " distinct clean skills.")

 ## Bag of words

Creating a **bag of words** means having all distinct words that form all the (clean) skills.

In [ ]:
def CreateBagOfWords(listOfStrings):
    '''
    Create a Pandas dataframe containing the individual words present in the skills and
    the amount of times they appear.
    Args:
    - listOfStrings: List with all the skills.
    Returns:
    - Pandas dataframe with columns "word" and "occurrences".
    '''
    bowDict = {}
    for string in listOfStrings:
        words = string.split(" ")
        for word in words:
            word = word.lstrip("(")
            word = word.rstrip(")")
            if word in bowDict.keys():
                bowDict[word] = bowDict[word] + 1
            else:
                bowDict[word] = 1
    bowDf = pd.DataFrame.from_dict(bowDict, orient = "index", columns = ["occurrences"])
    bowDf = pd.DataFrame.sort_values(bowDf, "occurrences", ascending = False).reset_index()
    bowDf = bowDf.rename(columns = {"index": "word"})
    return(bowDf)

bowDf = CreateBagOfWords(skills["skill_clean"])
bowDf

In [ ]:
wordsNum = len(set(bowDf["word"]))
print("Ths clean skills contain:")
print("- "+ format(wordsNum, ",") + " distinct words.")

## Creating a clustering model

I create a clustering model using the skills dataset. I will group together those offers that require similar skills.

### Train-test split

I split the data in train and test subsets. I will create the model with the train set and evaluate the best parameters for the model using the test set. After that, in the notebook that contains the prototype I will perform tests with unseen data.

In [ ]:
def TrainTestSplit(df, testSize = 0.8):
    '''
    Split dataset in train and test subsets.
    Args:
    - df: Pandas dataframe containing the dataset to split.
    - testSize: Portion of the ids in the test subset.
    Returns:
    - Two pandas dataframes with the two subsets.
    '''
    ids = pd.Series(list(set(df["id"])))
    idsLen = len(ids)
    np.random.seed(31)
    msk = np.random.rand(idsLen) < testSize
    trainIds = ids[msk]
    trainDf = df[df["id"].isin(trainIds)]
    testIds = ids[~msk]
    testDf = df[df["id"].isin(testIds)]
    return(trainDf.copy(), testDf.copy())

skillsTrain, skillsTest = TrainTestSplit(skills)

### Feature engineering

I create one-hot encoded features for the words with more occurrences. They are 1 if the word is present in at least one of the skills of the offer, an 0 otherwise. To show how it works I create features based on 10 categories.

In [ ]:
NUMBER_OF_CATEGORIES = 10

categories = bowDf.loc[0:NUMBER_OF_CATEGORIES - 1, "word"].tolist()

def FeatureEngineering(df, categories):
    '''
    Create one-hot encoded variables based on the provided categories.
    Args:
    - df: Pandas dataframe with a column called "skill_clean".
    - categories: List of words to create the features.
    Returns:
    - Pandas dataframe with every feature in a column.
    '''
    df = df.copy()
    for c in categories:
        df.insert(len(df.columns), c, df.skill_clean.apply(lambda x: int(c in x.split(" "))))
    dfId = df.groupby("id").agg("sum").reset_index()
    dfId.iloc[:,1:(NUMBER_OF_CATEGORIES + 1)] = (dfId.iloc[:,1:(NUMBER_OF_CATEGORIES + 1)] != 0) * 1
    dfId = dfId.rename(columns = {"index": "id"})
    return dfId

modelDf = FeatureEngineering(skillsTrain, categories)
modelDf

### Clustering using K-Means (first attempt)

I first try a K-Means clustering with K = 50; later I will explore which is the best number of clusters.

In [ ]:
NUMBER_OF_CLUSTERS = 50

array = np.array(modelDf.drop(["id"], axis = 1), dtype = np.float32)
model = KMeans(n_clusters = NUMBER_OF_CLUSTERS)
model = model.fit(array)

In [ ]:
modelDf["cluster"] = model.labels_
print("Number of offers per cluster:")
for n in range(NUMBER_OF_CLUSTERS):
    num = len(modelDf.loc[modelDf["cluster"] == n]["id"].tolist())
    print("Cluster: " + str(n) + ": " + str(num) + " offers.")

In [ ]:
CLUSTER = 20
mergeDf = pd.merge(skillsTrain[["id", "skill"]], modelDf[["id", "cluster"]], on = "id")
idsCluster = list(set(mergeDf.loc[mergeDf["cluster"] == CLUSTER]["id"].tolist()))
print("Examples of offers in the same cluster:")
for i in idsCluster[0:20]:
    print(set(mergeDf.loc[mergeDf["id"] == i]["skill"].tolist()))

### Finding the best parameters

To evaluate the goodness of the clustering, I calculate the mean distance of each offer to the centroid of the cluster it belongs to. I do that in the training set (used to calculate the clusters) and in the training set. The goal of this process is to find the best number of clusters.

I will be using 200 categories. This number shoud be tunned in order to find the optimal number of categories.

This step takes long time. For this reason, I added the `readFromFile` parameter in the `CalculateScoreForDifferentK` function. When `TRUE`, the resulting scores are read from a cached CSV file.

In [ ]:
NUMBER_OF_CATEGORIES = 200
NUMBER_OF_CLUSTERS_LIST = [10, 20, 50, 100, 200, 500, 1000]

def CalculateMeanDistance(df, model):
    '''
    Calculate the mean distance of each observation to the centroid of the cluster
    it belongs to.
    Args:
    - df: Pandas dataframe with a column called "cluster" and columns with features.
    - model: KMeans object containing the trained model.
    Returns:
    - Mean distance.
    '''
    df = df.copy()
    centers = model.cluster_centers_
    distance = 0
    for i in range(len(centers)):
        distances = (df.loc[df["cluster"] == i, categories] - np.array(centers[i])).pow(2).sum(1).pow(0.5)
        distance = distance + distances.sum()
    distance = distance / len(df.index)
    return distance

def CalculateScoreForDifferentK(df, kList, file = "scores.csv", readFromFile = False):
    '''
    Try different values of K for performing the KMeans and saves results in a CSV file.
    Args:
    - df: Pandas dataframe over which make the predictions (test).
    - KList: List of values of K to test.
    - file: File in which write the results or read them if readFromFile is True.
    - readFromFile: If True, results are read from file and computations are avoided.
    Returns:
    - Dataframe with results.
    '''
    if readFromFile:
        scoresDf = pd.read_csv(file)
        return scoresDf
    modelDfTrain, modelDfTest = TrainTestSplit(df)
    arrayTrain = np.array(modelDfTrain.drop(["id"], axis = 1), dtype = np.float32)
    arrayTest = np.array(modelDfTest.drop(["id"], axis = 1), dtype = np.float32)
    scoresDf = pd.DataFrame(columns = ["clusters", "score", "set"])
    for k in NUMBER_OF_CLUSTERS_LIST:
        print(str(k) + " clusters")

        model = KMeans(n_clusters = k)
        model = model.fit(arrayTrain)
        modelDfTrain.loc[:, "cluster"] = model.labels_

        predictions = model.predict(arrayTest)
        modelDfTest.loc[:, "cluster"] = predictions

        scoreTest = CalculateMeanDistance(modelDfTest, model)
        scoreTrain = CalculateMeanDistance(modelDfTrain, model)

        scoresDf = scoresDf.append({"clusters": k,
                                    "score": scoreTest,
                                    "set": "test"}, ignore_index = True)
        scoresDf = scoresDf.append({"clusters": k,
                                    "score": scoreTrain,
                                    "set": "train"}, ignore_index = True)
    scoresDf.to_csv(file, index = False)
    return scoresDf
    
categories = bowDf.loc[0:NUMBER_OF_CATEGORIES - 1, "word"].tolist()
modelDf = FeatureEngineering(skills, categories)
scoresDf = CalculateScoreForDifferentK(modelDf, NUMBER_OF_CLUSTERS_LIST, file = "scores.csv", readFromFile = True)
scoresDf

In [ ]:
dplot = scoresDf.copy()
dplot.set_index("clusters", inplace = True)
dplot.groupby("set")["score"].plot(legend = True)

In [ ]:
dplot.groupby("set")["score"].plot(legend = True, ylim = (0,2))

# Discussion

As expected, the larger the number of clusters, the lower the mean distance of observation to the center of its cluster. With this result, I will build a clustering model with 200 clusters. The number of clusters has been selected based on performance and time of execution. I know that more tests should be done to find the best model, including tunning other parameters and trying other algorithms.

# Other things I would do if I had more time

- Improving the feature engineering process using methods to obtain the optimal number of categories.
- Inculde data from the offers file to the model.
- Create categories by joining similar words.
- Use the distance between words instead of whether they are present or not.
- Using NLP to join different words with similar meaning.
- Trying other algorithms, in particular those in which an observation can fall in more than one cluster.
- Understand how the clustering is done in order to gain insights that can help improving the model.
- Performing PCA to reduce the number of features and speed up the process.
- Run tests using the offers in the offers data set.

The **recommender** notebook includes a working prototype consisting on a clustering model created based on the insights
presented here.